In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
df_1 = pd.read_csv('output.csv')
df_2 = pd.read_csv('submission_Caser_recall.csv')

In [3]:
users = df_1['user'].unique().tolist()

In [4]:
dict_target = {
    'user': list(),
    'item': list(),
}

In [5]:
def get_item(df, user, num_extract):
    return df[df['user']==user]['item'].to_numpy()[:num_extract]

In [6]:
topK = 10
num_least = 3
for user in tqdm(users):
    item_1 = df_1[df_1['user']==user]['item'].to_numpy() # 성능이 더 좋은 csv
    item_2 = df_2[df_2['user']==user]['item'].to_numpy()

    common = np.intersect1d(item_1, item_2)
    left_item_1 = np.setdiff1d(item_1, common)
    left_item_2 = np.setdiff1d(item_2, common)

    num_need = topK - len(common) 
    
    # 성능이 좋지 않은 csv(item_2)는 max(common, num_least)만큼만 포함한다.
    if len(common) > num_least:
        len_item_2 = 0
    else:
        len_item_2 = num_least - len(common)
        
    len_item_1 = topK - len(common) - len_item_2
    
    
    if len_item_2 > 0:
        items = np.concatenate((common, left_item_1[:len_item_1], left_item_2[:len_item_2]))
    else:
        items = np.concatenate((common, left_item_1[:len_item_1]))
    
    assert len(items) == 10, "concatenate error"
    
    dict_target['user'].extend([user]*10)
    dict_target['item'].extend(items)

100%|██████████| 31360/31360 [00:41<00:00, 748.46it/s]


In [7]:
df_target = pd.DataFrame(dict_target)

In [8]:
df_target.to_csv('ensemble.csv', index=False)